# Kotlin NDArrays

Thanks to **Multik** and the standard Collections library, it's easy to create, access and manipulate N-dimensional arrays and matrices.

You can import Multik using Jupyter's magic command `%use`, for importing Multik library and dependencies.

In [17]:
%use multik

We can create a simple 1D array as follows (type declaration is not mandatory)

In [47]:
val arr: NDArray<Int, D1> = mk.ndarray(mk[1, 2, 3])
arr

[1, 2, 3]

Notice the type definition of the Array, where we must specify:
- **Type**: Any Kotlin subtype of `Number` (`Boolean` is not permitted).
- **Dimension**: Multik defines 5 `Dimension` types: `D1`, `D2`, `D3`, `D4`, `DN`. `DN` is used in all the cases where the dimension is not known in advance, or when the dimension size is major than 4.

In the same way, we can create a 2 dimensional array, but this time we let the compiler infer the type and dimension

In [52]:
val arr = mk.ndarray(mk[mk[1, 2, 3], mk[4, 5, 6]])
arr

[[1, 2, 3],
[4, 5, 6]]

We can also create an array from *Kotlin Collections* **List** or **Set**

In [57]:
val arr = mk.ndarray(listOf(1, 2, 3))
arr

[1, 2, 3]

## Creation

We will go through simple `NDArrays` creation methods.

Creating equally spaced arrays with `arange` and `linspace`

In [158]:
val a = mk.linspace<Double>(0, 1, 5)
val b = mk.arange<Int>(0, 10, 2)

println("a -> $a")
println("b -> $b")

a -> [0.0, 0.25, 0.5, 0.75, 1.0]
b -> [0, 2, 4, 6, 8]


Generating an array of Zeros and Ones

In [101]:
val zrs = mk.zeros<Double>(10)
val ons = mk.ones<Double>(10)
println("Zeros ->$zrs")
println("Ones -> $ons")

Zeros ->[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Ones -> [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


We can then map the array to a matrix using the `reshape` method, wich takes in input an arbitrary number of dimensions size along wich the array will be mapped. 

In [102]:
mk.zeros<Int>(50).reshape(2, 5, 5) // three dimensional matrix with (z, x, y) = (2, 5, 5)

[[[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0]],

[[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0]]]

It's also possiible to create an `NDArray` providing a **lambda** for constructing it, considering that the matrix will be built upon a `arange` vector.

In [149]:
mk.d3array(2, 5, 5) { it % 2 }

[[[0, 1, 0, 1, 0],
[1, 0, 1, 0, 1],
[0, 1, 0, 1, 0],
[1, 0, 1, 0, 1],
[0, 1, 0, 1, 0]],

[[1, 0, 1, 0, 1],
[0, 1, 0, 1, 0],
[1, 0, 1, 0, 1],
[0, 1, 0, 1, 0],
[1, 0, 1, 0, 1]]]

Same as

In [164]:
mk.arange<Int>(50).map { it % 2 }.reshape(2, 5, 5)

[[[0, 1, 0, 1, 0],
[1, 0, 1, 0, 1],
[0, 1, 0, 1, 0],
[1, 0, 1, 0, 1],
[0, 1, 0, 1, 0]],

[[1, 0, 1, 0, 1],
[0, 1, 0, 1, 0],
[1, 0, 1, 0, 1],
[0, 1, 0, 1, 0],
[1, 0, 1, 0, 1]]]

TODO:
- real world scenarios
- linear algebra
- statics